---
description: API details.
output-file: machinery_calculation.html
title: module name here

---



In [ ]:
# | default_exp machinery_calculation
%load_ext autoreload
%autoreload 2

In [ ]:
# | include: false
from nbdev.showdoc import *

In [ ]:
# | export
from typing import List, Optional, Union, Type, TypeVar

import MachSysS.gymir_result_pb2 as proto_gymir
from MachSysS.convert_proto_timeseries import (
    convert_proto_timeseries_to_pd_dataframe,
    convert_proto_timeseries_for_multiple_propulsors_to_pd_dataframe,
)
import numpy as np
import pandas as pd
from feems.components_model.utility import IntegrationMethod
from feems.system_model import (
    ElectricPowerSystem,
    FuelOption,
    HybridPropulsionSystem,
    MechanicalPropulsionSystemWithElectricPowerSystem,
    FEEMSResultForMachinerySystem,
)
from feems.types_for_feems import FEEMSResult, TypePower
from feems.simulation_interface import SimulationInterface
from feems.fuel import FuelSpecifiedBy

from RunFeemsSim.pms_basic import (
    PmsLoadTable,
    get_min_load_table_dict_from_feems_system,
    PmsLoadTableSimulationInterface,
)

Numeric = TypeVar("Numeric", int, float, np.ndarray)


def convert_gymir_result_to_propulsion_power_series(
    gymir_result: proto_gymir.GymirResult,
) -> pd.Series:
    time = map(lambda each: each.epoch_s, gymir_result.result)
    power = map(lambda each: each.power_kw, gymir_result.result)
    return pd.Series(index=time, data=power)


class MachineryCalculation:
    def __init__(
        self,
        feems_system: Union[
            ElectricPowerSystem,
            MechanicalPropulsionSystemWithElectricPowerSystem,
            HybridPropulsionSystem,
        ],
        pms: SimulationInterface = None,
        maximum_allowed_power_source_load_percentage: float = 80,
    ):
        self.system_feems = feems_system
        if pms is None:
            load_table = PmsLoadTable(
                min_load2on_pattern=get_min_load_table_dict_from_feems_system(
                    system=feems_system,
                    maximum_allowed_genset_load_percentage=maximum_allowed_power_source_load_percentage,
                )
            )
            self.pms = PmsLoadTableSimulationInterface(
                n_bus_ties=1, pms_load_table=load_table
            )
        else:
            self.pms = pms
        self._set_equal_load_sharing_on_power_sources(n_datapoints=1)

    @property
    def electric_system(self) -> ElectricPowerSystem:
        if self.system_is_not_electric:
            return self.system_feems.electric_system
        else:
            return self.system_feems

    @property
    def system_is_not_electric(self) -> bool:
        return hasattr(self.system_feems, "electric_system")

    def _set_input_load_from_gymir_result(
        self,
        *,
        gymir_result: proto_gymir.GymirResult,
    ) -> None:
        propulsion_power_timeseries = convert_gymir_result_to_propulsion_power_series(
            gymir_result
        )
        self._set_input_load_time_interval_from_propulsion_power_time_series(
            propulsion_power_time_series=propulsion_power_timeseries,
            auxiliary_load_kw=gymir_result.auxiliary_load_kw,
        )

    def _set_input_load_time_interval_from_propulsion_power_time_series(
        self,
        *,
        propulsion_power_time_series: Union[pd.Series, pd.DataFrame],
        auxiliary_load_kw: Numeric,
        time_is_given_as_interval: bool = False,
    ) -> None:
        """Set the input load time interval from the propulsion power time series.
        Args:
            propulsion_power_time_series (Union[pd.Series, pd.DataFrame]): The propulsion power time series.
                If it is a DataFrame, it should contain the propulsion power for each propulsion drive.
            auxiliary_load_kw (Numeric): The auxiliary load in kW. It can be a single value or an array.
            time_is_given_as_interval (bool): If True, the time series index is given as intervals.
                Default is False, meaning the index is given as timestamps.
        """
        if not isinstance(propulsion_power_time_series, (pd.Series, pd.DataFrame)):
            raise TypeError(
                "propulsion_power_time_series must be a pandas Series or DataFrame."
            )
        if isinstance(propulsion_power_time_series, pd.Series):
            if time_is_given_as_interval:
                propulsion_power = propulsion_power_time_series.values
                time_interval_s = propulsion_power_time_series.index.to_numpy()
            else:
                propulsion_power = propulsion_power_time_series.values[:-1]
                time_interval_s = np.diff(propulsion_power_time_series.index.to_numpy())
            number_points = len(propulsion_power)
            # set power load
            if self.system_is_not_electric:
                number_of_propulsors = (
                    self.system_feems.mechanical_system.no_mechanical_loads
                )
            else:
                number_of_propulsors = len(self.electric_system.propulsion_drives)
            if self.system_is_not_electric:
                for propulsor in self.system_feems.mechanical_system.mechanical_loads:
                    propulsor.set_power_input_from_output(
                        propulsion_power / number_of_propulsors
                    )
            else:
                for propulsor in self.electric_system.propulsion_drives:
                    propulsor.set_power_input_from_output(
                        propulsion_power / number_of_propulsors
                    )
        else:
            if time_is_given_as_interval:
                time_interval_s = propulsion_power_time_series.index.to_numpy()
            else:
                time_interval_s = np.diff(propulsion_power_time_series.index.to_numpy())
                propulsion_power_time_series = propulsion_power_time_series.iloc[:-1]
            number_points = len(propulsion_power_time_series)
            for propulsor_name in propulsion_power_time_series.columns:
                propulsor = None
                if self.system_is_not_electric:
                    for (
                        each_shaft_line
                    ) in self.system_feems.mechanical_system.shaft_line:
                        try:
                            propulsor = (
                                each_shaft_line.get_component_by_name_power_type(
                                    name=propulsor_name,
                                    power_type=TypePower.POWER_CONSUMER,
                                )
                            )
                        except ValueError:
                            continue
                else:
                    for (
                        swb_id,
                        each_switchboard,
                    ) in self.electric_system.switchboards.items():
                        try:
                            propulsor = (
                                each_switchboard._get_component_by_type_and_name(
                                    name=propulsor_name,
                                    power_type=TypePower.POWER_CONSUMER,
                                )
                            )
                        except ValueError:
                            continue
                if propulsor is None:
                    raise ValueError(
                        f"Propulsor with name {propulsor_name} not found in the system."
                    )
                propulsor.set_power_input_from_output(
                    propulsion_power_time_series[propulsor_name].values
                )
        auxiliary_load_kw = np.atleast_1d(auxiliary_load_kw)
        if len(auxiliary_load_kw) > 1:
            auxiliary_load_kw = auxiliary_load_kw[:number_points]
        else:
            auxiliary_load_kw = np.repeat(auxiliary_load_kw, number_points)
        number_of_other_loads = len(self.electric_system.other_load)
        if number_of_other_loads == 0:
            assert np.all(
                np.atleast_1d(auxiliary_load_kw) == 0
            ), "Auxiliary load is not zero while other loads are not defined in the system."
        for other_load in self.electric_system.other_load:
            other_load.power_input = auxiliary_load_kw / number_of_other_loads
        self.electric_system.set_time_interval(
            time_interval_s=time_interval_s,
            integration_method=IntegrationMethod.sum_with_time,
        )
        if self.system_is_not_electric:
            self.system_feems.mechanical_system.set_time_interval(
                time_interval_s=time_interval_s,
                integration_method=IntegrationMethod.sum_with_time,
            )

    def _set_equal_load_sharing_on_power_sources(self, n_datapoints: int) -> None:
        for power_source in self.electric_system.power_sources:
            power_source.load_sharing_mode = np.zeros(shape=[n_datapoints])
        if self.system_is_not_electric:
            for power_source in self.system_feems.mechanical_system.main_engines:
                power_source.load_sharing_mode = np.zeros(shape=[n_datapoints])

    def _set_status_for_mechanical_system(self) -> None:
        """Set the status of main engines for the mechanical system, turning all the gensets on.
        The main engines that are not used for propulsion after the power balance calculation
        will be turned off.
        """
        if np.isscalar(self.system_feems.mechanical_system.time_interval_s):
            n_data_points = 1
        else:
            n_data_points = len(self.system_feems.mechanical_system.time_interval_s)
        for main_engine in self.system_feems.mechanical_system.main_engines:
            main_engine.status = np.ones(n_data_points).astype(bool)

    def _run_simulation(
        self,
        fuel_specified_by: FuelSpecifiedBy = FuelSpecifiedBy.IMO,
        ignore_power_balance: bool = False,
        fuel_option: Optional[FuelOption] = None,
    ) -> Union[FEEMSResult, FEEMSResultForMachinerySystem]:
        """Run the simulation and return the result.

        Args:
            fuel_specified_by(FuelSpecifiedBy): The fuel specified by IMO/EU/USER. Default is IMO.
            ignore_power_balance(bool): If True, the power balance calculation will be ignored.
            fuel_option(FuelOption): The fuel option to be used in the simulation. If None, the
                default fuel option in the system will be used.

        Returns:
            The result of the simulation. FEEMSResult or FEEMSResultForMachinery system.
        """
        if ignore_power_balance:
            if self.system_is_not_electric:
                return self.system_feems.get_fuel_energy_consumption_running_time(
                    time_interval_s=self.system_feems.mechanical_system.time_interval_s,
                    integration_method=IntegrationMethod.sum_with_time,
                    fuel_specified_by=fuel_specified_by,
                    fuel_option=fuel_option,
                )
            else:
                return self.system_feems.get_fuel_energy_consumption_running_time(
                    fuel_specified_by=fuel_specified_by,
                    fuel_option=fuel_option,
                )

        power_kw_per_switchboard = (
            self.electric_system.get_sum_consumption_kw_sources_switchboard()
        )
        self.pms.set_status(
            power_kw_per_switchboard=power_kw_per_switchboard,
            electric_power_system=self.electric_system,
            time_interval_s=self.electric_system.time_interval_s,
            power_source_priority=None,
        )
        if self.system_is_not_electric:
            self._set_status_for_mechanical_system()
            self.system_feems.do_power_balance_calculation()
            return self.system_feems.get_fuel_energy_consumption_running_time(
                time_interval_s=self.system_feems.mechanical_system.time_interval_s,
                integration_method=IntegrationMethod.sum_with_time,
                fuel_specified_by=fuel_specified_by,
                fuel_option=fuel_option,
            )
        else:
            self.system_feems.do_power_balance_calculation()
            return self.system_feems.get_fuel_energy_consumption_running_time(
                fuel_specified_by=fuel_specified_by,
                fuel_option=fuel_option,
            )

    def calculate_machinery_system_output_from_gymir_result(
        self,
        *,
        gymir_result: proto_gymir.GymirResult,
        fuel_specified_by: FuelSpecifiedBy = FuelSpecifiedBy.IMO,
        ignore_power_balance: bool = False,
        fuel_option: Optional[FuelOption] = None,
    ) -> Union[FEEMSResult, FEEMSResultForMachinerySystem]:
        """
        Calculate the machinery system output from a Gymir result.

        Args:
            gymir_result(GymirResult): Gymir result given as protobuf message.
            fuel_specified_by(FuelSpecifiedBy): The fuel specified by IMO/EU/USER. Default is IMO.
            ignore_power_balance(bool): If True, the power balance calculation will be ignored.
            fuel_option(FuelOption): The fuel option to be used in the simulation. If None, the
                default fuel option in the system will be used.

        Returns:
            The result of the calculation. FEEMSResult or FEEMSResultForMachinerySystem.
        """
        self._set_input_load_from_gymir_result(gymir_result=gymir_result)
        return self._run_simulation(
            fuel_specified_by=fuel_specified_by,
            ignore_power_balance=ignore_power_balance,
            fuel_option=fuel_option,
        )

    def calculate_machinery_system_output_from_propulsion_power_time_series(
        self,
        *,
        propulsion_power: Union[pd.Series, pd.DataFrame],
        auxiliary_power_kw: Numeric,
        fuel_specified_by: FuelSpecifiedBy = FuelSpecifiedBy.IMO,
        ignore_power_balance: bool = False,
        fuel_option: Optional[FuelOption] = None,
    ) -> Union[FEEMSResult, FEEMSResultForMachinerySystem]:
        """
        Calculate the machinery system output from a time series of the propulsion power and
        auxiliary power.

        Args:
            propulsion_power: The propulsion power time series. It can be a pandas Series if the
                data is the total propulsion power that is equally shared by all propulsion drives,
                or a pandas DataFrame if the data is given for each propulsion drive separately.
            auxiliary_power_kw(Numeric): The auxiliary power in kW. It can be a single value or
                a numpy array with the same length as the propulsion power.
            fuel_specified_by(FuelSpecifiedBy): The fuel specified by IMO/EU/USER. Default is IMO.
            ignore_power_balance(bool): If True, the power balance calculation will be ignored.
            fuel_option(FuelOption): The fuel option to be used in the simulation. If None, the
                default fuel option in the system will be used.

        Returns:
            The result of the calculation. FEEMSResult or FEEMSResultForMachinerySystem.
        """
        # Check if the propulsion power is a pandas Series or DataFrame
        if not isinstance(propulsion_power, (pd.Series, pd.DataFrame)):
            raise TypeError("propulsion_power must be a pandas Series or DataFrame.")
        # If it's a DataFrame, ensure column names match the propulsion drives
        if isinstance(propulsion_power, pd.DataFrame):
            # Check for electric propulsion system
            if self.system_is_not_electric:
                names_for_propulsion_drives = [
                    drive.name
                    for drive in self.system_feems.mechanical_system.mechanical_loads
                ]
                for name in propulsion_power.columns:
                    assert (
                        name in names_for_propulsion_drives
                    ), f"Column '{name}' not found in mechanical loads."
            else:
                names_for_propulsion_drives = [
                    drive.name for drive in self.electric_system.propulsion_drives
                ]
                for name in propulsion_power.columns:
                    assert (
                        name in names_for_propulsion_drives
                    ), f"Column '{name}' not found in propulsion drives."
        # Check if auxiliary_power_kw is a scalar or an array
        if not np.isscalar(auxiliary_power_kw):
            assert (
                len(propulsion_power) == len(auxiliary_power_kw)
                or len(auxiliary_power_kw) == 1
            ), f"The length of the auxiliary power({len(auxiliary_power_kw)}) must be 1 or the same as the propulsion power ({len(propulsion_power)})"

        self._set_input_load_time_interval_from_propulsion_power_time_series(
            propulsion_power_time_series=propulsion_power,
            auxiliary_load_kw=auxiliary_power_kw,
        )
        return self._run_simulation(
            fuel_specified_by=fuel_specified_by,
            ignore_power_balance=ignore_power_balance,
            fuel_option=fuel_option,
        )

    def calculate_machinery_system_output_from_time_series_result(
        self,
        *,
        time_series: Union[
            proto_gymir.TimeSeriesResult,
            proto_gymir.TimeSeriesResultForMultiplePropulsors,
        ],
        fuel_specified_by: FuelSpecifiedBy = FuelSpecifiedBy.IMO,
        ignore_power_balance: bool = False,
        fuel_option: Optional[FuelOption] = None,
    ) -> Union[FEEMSResult, FEEMSResultForMachinerySystem]:
        """
        Calculate the machinery system output from statistics of the propulsion power.
        Args:
            time_series(TimeSeriesResult): Time series result given as protobuf message.
            fuel_specified_by(FuelSpecifiedBy): The fuel specified by IMO/EU/USER. Default is IMO.
            ignore_power_balance(bool): If True, the power balance calculation will be ignored.
            fuel_option(FuelOption): The fuel option to be used in the simulation. If None, the
                default fuel option in the system will be used.

        Returns:
            The result of the simulation. FEEMSResult or FEEMSResultForMachinery system.
        """
        if isinstance(time_series, proto_gymir.TimeSeriesResult):
            df = convert_proto_timeseries_to_pd_dataframe(time_series)
            self._set_input_load_time_interval_from_propulsion_power_time_series(
                propulsion_power_time_series=df["propulsion_power_kw"],
                auxiliary_load_kw=df["auxiliary_power_kw"].values,
            )
        elif isinstance(time_series, proto_gymir.TimeSeriesResultForMultiplePropulsors):
            df = convert_proto_timeseries_for_multiple_propulsors_to_pd_dataframe(
                time_series
            )
            propulsor_names = list(map(lambda each: each, time_series.propulsor_names))
            self._set_input_load_time_interval_from_propulsion_power_time_series(
                propulsion_power_time_series=df[propulsor_names],
                auxiliary_load_kw=df["auxiliary_power_kw"].values,
            )
        else:
            raise TypeError(
                "time_series must be a TimeSeriesResult or TimeSeriesResultForMultiplePropulsors."
            )

        return self._run_simulation(
            fuel_specified_by=fuel_specified_by,
            ignore_power_balance=ignore_power_balance,
            fuel_option=fuel_option,
        )

    def calculate_machinery_system_output_from_statistics(
        self,
        *,
        propulsion_power: np.ndarray,
        frequency: np.ndarray,
        auxiliary_power_kw: Numeric,
        fuel_specified_by: FuelSpecifiedBy = FuelSpecifiedBy.IMO,
        ignore_power_balance: bool = False,
        fuel_option: Optional[FuelOption] = None,
    ) -> Union[FEEMSResult, FEEMSResultForMachinerySystem]:
        """
        Calculate the machinery system output from statistics of the propulsion power.

        Args:
            propulsion_power(np.ndarray): The propulsion power for each mode in kW.
            frequency(np.ndarray): The frequency of each mode in seconds. If the frequency is
                given as normalized value, the output should be interpreted as per second value.
            auxiliary_power_kw(Numeric): The auxiliary power for each mode in kW. It is also
                possible to give a single value for all modes.
            fuel_specified_by(FuelSpecifiedBy): The fuel specified by IMO/EU/USER. Default is IMO.
            ignore_power_balance(bool): If True, the power balance calculation will be ignored.
            fuel_option(FuelOption): The fuel option to be used in the simulation. If None, the
                default fuel option in the system will be used.

        Returns:
            The result of the simulation. FEEMSResult or FEEMSResultForMachinery system.
        """
        if not np.isscalar(auxiliary_power_kw):
            assert (
                len(propulsion_power) == len(auxiliary_power_kw)
                or len(auxiliary_power_kw) == 1
            ), "The length of the auxiliary power must be 1 or the same as the propulsion power"
        self._set_input_load_time_interval_from_propulsion_power_time_series(
            propulsion_power_time_series=pd.Series(
                data=propulsion_power, index=frequency
            ),
            auxiliary_load_kw=auxiliary_power_kw,
            time_is_given_as_interval=True,
        )
        return self._run_simulation(
            fuel_specified_by=fuel_specified_by,
            ignore_power_balance=ignore_power_balance,
            fuel_option=fuel_option,
        )

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import MachSysS.system_structure_pb2 as proto_system
from MachSysS.gymir_result_pb2 import (
    GymirResult,
    SimulationInstance,
    TimeSeriesResult,
    TimeSeriesResultForMultiplePropulsors,
    PropulsionPowerInstanceForMultiplePropulsors,
)
from MachSysS.convert_to_feems import convert_proto_propulsion_system_to_feems
from MachSysS.utility import retrieve_machinery_system_from_file
from MachSysS.convert_feems_result_to_proto import FEEMSResultConverter
from RunFeemsSim.machinery_calculation import (
    MachineryCalculation,
    convert_gymir_result_to_propulsion_power_series,
)
from feems.components_model.component_mechanical import (
    EngineMultiFuel,
    MainEngineForMechanicalPropulsion,
    FuelCharacteristics,
)
from feems.fuel import TypeFuel, FuelOrigin, FuelSpecifiedBy
from feems.types_for_feems import TypePower, TypeComponent, EngineCycleType
from feems.components_model.component_electric import Genset
from feems.system_model import FuelOption

pd.options.plotting.backend = "plotly"

In [ ]:
def create_gymir_result() -> GymirResult:
    return GymirResult(
        name="test",
        auxiliary_load_kw=random.random() * 500,
        result=[
            SimulationInstance(epoch_s=100 * index + 1, power_kw=2000 * random.random())
            for index, _ in enumerate(range(10))
        ],
    )


# Testing convert gymir result to propulsion power series
gymir_result = create_gymir_result()
propulsion_power = convert_gymir_result_to_propulsion_power_series(gymir_result)
assert len(gymir_result.result) == len(propulsion_power)
propulsion_power.plot()

# Testing MachineryCalculation
save_the_result = False
path = os.path.join("tests", "system_proto.mss")
system_proto = retrieve_machinery_system_from_file(path)
print(system_proto)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)
machinery_calculation = MachineryCalculation(feems_system=system_feems)
res = machinery_calculation.calculate_machinery_system_output_from_gymir_result(
    gymir_result=gymir_result
)
if save_the_result:
    with open("system_proto.mss", "wb") as file:
        file.write(system_proto.SerializeToString())
average_power_consumption = 0
running_hours_mean = 0
for power_source in machinery_calculation.electric_system.power_sources:
    running_hours_mean += (power_source.power_output > 0).mean()
for propulsor in machinery_calculation.electric_system.propulsion_drives:
    average_power_consumption += propulsor.power_input.mean()
average_bsfc_for_engines = res.fuel_consumption_total_kg / (
    res.duration_s / 3600000 * average_power_consumption / 0.95
)
print(f"Average bsfc for engines: {average_bsfc_for_engines} g/kWh")
assert 200 < average_bsfc_for_engines < 385

name: "Electric propulsion system"
propulsion_type: ELECTRIC
maximum_allowed_genset_load_percentage: 80
electric_system {
  switchboards {
    switchboard_id: 1
    subsystems {
      engine {
        rated_power_kw: 1000
        rated_speed_rpm: 1000
        bsfc {
          curve {
            x_label: "power load"
            y_label: "bsfc"
            curve {
              points {
                x: 0.1
                y: 270
              }
              points {
                x: 0.25
                y: 240
              }
              points {
                x: 0.5
                y: 220
              }
              points {
                x: 0.75
                y: 210
              }
              points {
                x: 1
                y: 220
              }
            }
          }
        }
        main_fuel {
          fuel_origin: FOSSIL
        }
        order_from_switchboard_or_shaftline: 2
        nox_calculation_method: TIER_3
        engine_cycle_type:

In [ ]:
# Testing MachineryCalculation with time series
auxiliary_power_kw = 100
res_scalar_aux = machinery_calculation.calculate_machinery_system_output_from_propulsion_power_time_series(
    propulsion_power=propulsion_power, auxiliary_power_kw=auxiliary_power_kw
)
print(len(propulsion_power))
auxiliary_power_kw = np.ones(len(propulsion_power)) * 100
res_array_aux = machinery_calculation.calculate_machinery_system_output_from_propulsion_power_time_series(
    propulsion_power=propulsion_power, auxiliary_power_kw=auxiliary_power_kw
)

# Testing MachineryCalculation with time series result in dataframe format
propulsor_names = [
    propulsion_drive.name for propulsion_drive in system_feems.propulsion_drives
]
data = np.array(
    [propulsion_power.values / len(propulsor_names) for _ in propulsor_names]
)
propulsion_power_df = pd.DataFrame(
    data=data.T,
    index=propulsion_power.index,
    columns=propulsor_names,
)
res_df = machinery_calculation.calculate_machinery_system_output_from_propulsion_power_time_series(
    propulsion_power=propulsion_power_df,
    auxiliary_power_kw=auxiliary_power_kw,
)

# Testing MachineryCalculation with time series result in proto format
time_series = TimeSeriesResultForMultiplePropulsors(
    propulsion_power_timeseries=[
        PropulsionPowerInstanceForMultiplePropulsors(
            epoch_s=index,
            propulsion_power_kw=row.values.tolist(),
            auxiliary_power_kw=each_aux_power,
        )
        for (index, row), each_aux_power in zip(
            propulsion_power_df.iterrows(), auxiliary_power_kw
        )
    ],
    propulsor_names=propulsion_power_df.columns.tolist(),
)
res_proto = (
    machinery_calculation.calculate_machinery_system_output_from_time_series_result(
        time_series=time_series,
        fuel_specified_by=FuelSpecifiedBy.IMO,
    )
)

assert np.allclose(
    res_scalar_aux.fuel_consumption_total_kg,
    res_array_aux.fuel_consumption_total_kg,
), "Fuel consumption for scalar and array auxiliary power should be the same."

assert np.allclose(
    res_scalar_aux.fuel_consumption_total_kg,
    res_df.fuel_consumption_total_kg,
), "Fuel consumption for scalar auxiliary power and dataframe should be the same."

assert np.allclose(
    res_scalar_aux.fuel_consumption_total_kg,
    res_proto.fuel_consumption_total_kg,
), "Fuel consumption for scalar auxiliary power and proto should be the same."

10


In [ ]:
# Testing MachineryCalculation with time series result (protobuf)
path_to_system = os.path.join("tests", "mechanical_propulsion_with_electric_system.mss")
system_proto = retrieve_machinery_system_from_file(path_to_system)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)

path_to_result = os.path.join("tests", "time_series_result.pb")
with open(path_to_result, "rb") as file:
    time_series_result = TimeSeriesResult()
    time_series_result.ParseFromString(file.read())

machinery_calculation = MachineryCalculation(feems_system=system_feems)
res = machinery_calculation.calculate_machinery_system_output_from_time_series_result(
    time_series=time_series_result
)

In [ ]:
total_energy_consumption_kwh = (
    np.array(
        [
            [prop_power_inst.propulsion_power_kw, prop_power_inst.auxiliary_power_kw]
            for prop_power_inst in time_series_result.propulsion_power_timeseries
        ]
    )
    * 60
    / 3600
)
total_propulsion_energy_consumption_kwh = np.sum(total_energy_consumption_kwh[:, 0])
total_auxiliary_energy_consumption_kwh = np.sum(total_energy_consumption_kwh[:, 1])
average_bsfc_for_main_engine = (
    res.mechanical_system.multi_fuel_consumption_total_kg.natural_gas
    / total_propulsion_energy_consumption_kwh
    * 1000
)
average_pilot_bsfc_for_main_engine = (
    res.mechanical_system.multi_fuel_consumption_total_kg.diesel
    / total_propulsion_energy_consumption_kwh
    * 1000
)
average_bsfc_for_auxiliary_engines = (
    res.electric_system.multi_fuel_consumption_total_kg.natural_gas
    / total_auxiliary_energy_consumption_kwh
    * 1000
    * 0.95
)
print(f"Average bsfc for main engine: {average_bsfc_for_main_engine} g/kWh")
print(
    f"Average bsfc for pilot fuel for main engine: {average_pilot_bsfc_for_main_engine} g/kWh"
)
print(f"Average bsfc for auxiliary engines: {average_bsfc_for_auxiliary_engines} g/kWh")
assert (
    150 > average_bsfc_for_main_engine > 140
), "Average bsfc for main engine is not within expected range"
assert (
    1.0 > average_pilot_bsfc_for_main_engine > 0.7
), "Average pilot bsfc for main engine is not within expected range"
assert (
    224 > average_bsfc_for_auxiliary_engines > 210
), "Average bsfc for auxiliary engines is not within expected range"

Average bsfc for main engine: 142.9406492665165 g/kWh
Average bsfc for pilot fuel for main engine: 0.886793584666771 g/kWh
Average bsfc for auxiliary engines: 213.19619694076079 g/kWh


In [ ]:
# Testing MachineryCalculation with time series result as dataframe (protobuf)
path_to_system = os.path.join("tests", "mechanical_propulsion_with_electric_system.mss")
system_proto = retrieve_machinery_system_from_file(path_to_system)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)

path_to_result = os.path.join("tests", "time_series_result.pb")
with open(path_to_result, "rb") as file:
    time_series_result = TimeSeriesResult()
    time_series_result.ParseFromString(file.read())


machinery_calculation = MachineryCalculation(feems_system=system_feems)
res = machinery_calculation.calculate_machinery_system_output_from_time_series_result(
    time_series=time_series_result
)

In [ ]:
# Testing MachineryCalculation for hybrid system with time series result as dataframe (protobuf)
path_to_system = os.path.join("tests", "machinery_system_hybrid.mss")
system_proto = retrieve_machinery_system_from_file(path_to_system)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)

path_to_result = os.path.join("tests", "time_series_result.pb")
with open(path_to_result, "rb") as file:
    time_series_result = TimeSeriesResult()
    time_series_result.ParseFromString(file.read())


machinery_calculation = MachineryCalculation(feems_system=system_feems)
res = machinery_calculation.calculate_machinery_system_output_from_time_series_result(
    time_series=time_series_result
)

In [ ]:
# Create a multi-fuel engine for the main engine
# Testing MachineryCalculation with time series result as dataframe (protobuf)
path_to_system = os.path.join("tests", "mechanical_propulsion_with_electric_system.mss")
system_proto = retrieve_machinery_system_from_file(path_to_system)
system_feems = convert_proto_propulsion_system_to_feems(system_proto)
main_engine = system_feems.mechanical_system.main_engines[0]
bsfc_vlsfo = main_engine.engine.specific_fuel_consumption_points.copy()
bsfc_vlsfo[:, 1] *= 48 / 41  # Assume 41.7 MJ/kg for VLSFO
multi_fuel_main_engine = EngineMultiFuel(
    type_=TypeComponent.MAIN_ENGINE,
    name=main_engine.engine.name,
    rated_power=main_engine.engine.rated_power,
    rated_speed=main_engine.engine.rated_speed,
    multi_fuel_characteristics=[
        FuelCharacteristics(
            main_fuel_type=TypeFuel.NATURAL_GAS,
            main_fuel_origin=FuelOrigin.FOSSIL,
            pilot_fuel_type=TypeFuel.DIESEL,
            pilot_fuel_origin=FuelOrigin.FOSSIL,
            bsfc_curve=main_engine.engine.specific_fuel_consumption_points,
            bspfc_curve=main_engine.engine.specific_pilot_fuel_consumption_points,
            engine_cycle_type=EngineCycleType.OTTO,
        ),
        FuelCharacteristics(
            main_fuel_type=TypeFuel.VLSFO,
            main_fuel_origin=FuelOrigin.FOSSIL,
            bsfc_curve=bsfc_vlsfo,
            engine_cycle_type=EngineCycleType.OTTO,
        ),
    ],
)
main_engine.engine = multi_fuel_main_engine

multi_fuel_gensets = []
aux_engine = system_feems.electric_system.power_sources[0].aux_engine
bspfc_curve = np.array([[0, 0], [0.25, 1.7], [0.5, 1.2], [0.75, 0.8], [1.0, 0.6]])
bsfc_vlsfo = aux_engine.specific_fuel_consumption_points.copy()
bsfc_vlsfo[:, 1] *= 48 / 41
for power_source in system_feems.electric_system.power_sources:
    if power_source.type == TypeComponent.GENSET:
        genset: Genset = power_source  # type: ignore
        multi_fuel_genset = EngineMultiFuel(
            type_=TypeComponent.AUXILIARY_ENGINE,
            name=genset.name,
            rated_power=genset.aux_engine.rated_power,
            rated_speed=genset.aux_engine.rated_speed,
            multi_fuel_characteristics=[
                FuelCharacteristics(
                    main_fuel_type=TypeFuel.NATURAL_GAS,
                    main_fuel_origin=FuelOrigin.FOSSIL,
                    pilot_fuel_type=TypeFuel.DIESEL,
                    pilot_fuel_origin=FuelOrigin.FOSSIL,
                    bsfc_curve=genset.aux_engine.specific_fuel_consumption_points,
                    bspfc_curve=bspfc_curve,
                    engine_cycle_type=EngineCycleType.OTTO,
                ),
                FuelCharacteristics(
                    main_fuel_type=TypeFuel.VLSFO,
                    main_fuel_origin=FuelOrigin.FOSSIL,
                    bsfc_curve=bsfc_vlsfo,
                    engine_cycle_type=EngineCycleType.DIESEL,
                ),
            ],
        )
        genset.aux_engine = multi_fuel_genset


# Test the fuel option argument in the methods of MachineryCalculation
machinery_calculation = MachineryCalculation(feems_system=system_feems)
res_no_option_imo = (
    machinery_calculation.calculate_machinery_system_output_from_time_series_result(
        time_series=time_series_result,
        fuel_specified_by=FuelSpecifiedBy.IMO,
    )
)
# Assert that there is no VLSFO consumption
assert (
    list(
        filter(
            lambda fuel: fuel.fuel_type == TypeFuel.VLSFO,
            res_no_option_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels,
        )
    )
    == []
), "VLSFO consumption should be zero when no fuel option is specified."

res_VLSFO_imo = (
    machinery_calculation.calculate_machinery_system_output_from_time_series_result(
        time_series=time_series_result,
        fuel_specified_by=FuelSpecifiedBy.IMO,
        fuel_option=FuelOption(fuel_type=TypeFuel.VLSFO, fuel_origin=FuelOrigin.FOSSIL, for_pilot=False, primary=False),
    )
)
# Assert that there is no natural gas consumption
assert (
    list(
        filter(
            lambda fuel: fuel.fuel_type == TypeFuel.NATURAL_GAS,
            res_VLSFO_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels,
        )
    )
    == []
), "Natural gas consumption should be zero when no fuel option is specified."

res_NATURAL_GAS_imo = (
    machinery_calculation.calculate_machinery_system_output_from_time_series_result(
        time_series=time_series_result,
        fuel_specified_by=FuelSpecifiedBy.IMO,
        fuel_option=FuelOption(
            fuel_type=TypeFuel.NATURAL_GAS, fuel_origin=FuelOrigin.FOSSIL, for_pilot=False, primary=True,
        ),
    )
)
# Assert that there is no natural gas consumption
assert (
    list(
        filter(
            lambda fuel: fuel.fuel_type == TypeFuel.VLSFO,
            res_NATURAL_GAS_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels,
        )
    )
    == []
), "Natural gas consumption should be zero when no fuel option is specified."

In [ ]:
print("Total fuel energy consumption for mechanical system in tons:")
total_energy_consumption_default_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_no_option_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_vlsfo_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_VLSFO_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_natural_gas_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_NATURAL_GAS_imo.mechanical_system.multi_fuel_consumption_total_kg.fuels
    ]
)
print(f" - Default Fuel (LNG) : {total_energy_consumption_default_gj / 1000:.2f}")
print(f" - VLSFO : {total_energy_consumption_vlsfo_gj / 1000:.2f}")
print(f" - Natural Gas : {total_energy_consumption_natural_gas_gj / 1000:.2f}")
assert (
    total_energy_consumption_default_gj == total_energy_consumption_natural_gas_gj
), "Total energy consumption should be the same for different fuel options."
assert (
    abs(total_energy_consumption_default_gj - total_energy_consumption_vlsfo_gj)
    / total_energy_consumption_default_gj
    < 1e-2
), "Total energy consumption should be the same for different fuel options."
print("Total fuel consumption for electric system in tons:")
total_energy_consumption_default_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_no_option_imo.electric_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_vlsfo_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_VLSFO_imo.electric_system.multi_fuel_consumption_total_kg.fuels
    ]
)
total_energy_consumption_natural_gas_gj = sum(
    [
        fuel.mass_or_mass_fraction * fuel.lhv_mj_per_g
        for fuel in res_NATURAL_GAS_imo.electric_system.multi_fuel_consumption_total_kg.fuels
    ]
)
print(f" - Default Fuel (LNG) : {total_energy_consumption_default_gj / 1000:.2f}")
print(f" - VLSFO : {total_energy_consumption_vlsfo_gj / 1000:.2f}")
print(f" - Natural Gas : {total_energy_consumption_natural_gas_gj / 1000:.2f}")
assert (
    total_energy_consumption_default_gj == total_energy_consumption_natural_gas_gj
), "Total energy consumption should be the same for different fuel options."
assert (
    abs(total_energy_consumption_default_gj - total_energy_consumption_vlsfo_gj)
    / total_energy_consumption_default_gj
    < 1e-2
), "Total energy consumption should be the same for different fuel options."
print("GHG Emissions in for mechanical system tons:")
print(
    f" - Default Fuel (LNG) : {res_no_option_imo.mechanical_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - VLSFO : {res_VLSFO_imo.mechanical_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - Natural Gas : {res_NATURAL_GAS_imo.mechanical_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)

print("GHG Emissions in for electric system tons:")
print(
    f" - Default Fuel (LNG) : {res_no_option_imo.electric_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - VLSFO : {res_VLSFO_imo.electric_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)
print(
    f" - Natural Gas : {res_NATURAL_GAS_imo.electric_system.co2_emission_total_kg.tank_to_wake_kg_or_gco2eq_per_gfuel / 1000:.2f}"
)

Total fuel energy consumption for mechanical system in tons:
 - Default Fuel (LNG) : 144.89
 - VLSFO : 144.10
 - Natural Gas : 144.89
Total fuel consumption for electric system in tons:
 - Default Fuel (LNG) : 27.05
 - VLSFO : 26.91
 - Natural Gas : 27.05
GHG Emissions in for mechanical system tons:
 - Default Fuel (LNG) : 8315.19
 - VLSFO : 11267.58
 - Natural Gas : 8315.19
GHG Emissions in for electric system tons:
 - Default Fuel (LNG) : 1552.26
 - VLSFO : 2104.61
 - Natural Gas : 1552.26
